# Detect extrusions (or another event) on a movie

Used already trained neural network(s) to detect extrusions or other events from movie(s).

DeXtrusion calculates a probability map which reflects the probability of having an extrusion (or other event) for each pixel at each time of the movie. The probability map can be converted to single point position as ImageJ ROI. The position of an event will be at the centroid of a high probability volume.

## Initialization

In [ ]:
import os
from dextrusion.DeXtrusion import DeXtrusion
from dextrusion.DialogDeXtrusion import DialogDeXtrusion

# default parameters
talkative = True  ## print info messages
dexter = DeXtrusion(verbose=talkative)

imname = "../data/notum_mix"    ## folder where data are saved to initialize the file browser
modeldir = os.path.join(os.path.abspath(os.path.join(os.getcwd(), os.pardir)), "DeXNets", "notum_all")  ## folder where models are saved to initialize the file browser
groupsize = 150000       ## number of windows that are runned through the neural network at the same time - depends on the computer processing capabilities
                         ## higher will run faster, but too high can fail

### Graphical interface pop-up to choose the detection parameters

In [ ]:
# Choose parameters through graphical interface
diag = DialogDeXtrusion()
diag.dialog_main(modeldir, imname)

## typical cell diameter in the movie. Used to resize the movie if necessary to match with the reference scale used in the training.
cell_diameter = float(diag.cell_diameter)          
## typical duration of an extrusion event (number of frames it is visible). Used to resize the movie to the reference scale if necessary
extrusion_duration = float(diag.extrusion_duration) 
## xy step used to move the sliding windows
shift_xy = int(diag.dxyval)
## frame interval used to move the sliding windows
shift_t = int(diag.dtval)
## save the resulting probability maps (of all events)
save_proba = bool(diag.saveProba)
## save the cleaned (thresholded) probability map of the selected event
save_proba_one = bool(diag.saveProbaOne)
## save the ROIs file of all the events
save_rois = bool(diag.saveRois)
## Choose which event to save the probamap of. (Put None to do all events)
cat = int(diag.event_ind)
## Names of the possible events
catnames = diag.event
## min average probability in volume to keep ROI
roi_threshold = float(diag.threshold) 
## min positive volume to keep ROI
roi_volume = float(diag.proba_vol) 
## computational parameters, number of windows to evaluate at the time
groupsize = int(diag.group_size)       

modeldir = diag.modeldir
from glob import glob
# if folder "modeldir" is not one model directory but contains several models subdirectory, use them all
if not os.path.exists( os.path.join(modeldir,"config.cfg") ):
    model = glob(modeldir+"/*/", recursive = False)
else:
    model = [modeldir]
imname = diag.imagepath
if talkative:
    print("Using network(s) "+str(model))
    print("On movie(s) "+str(imname))

### Run the dextrusion detection on the selected movie(s) with the selected trained network(s)

In [ ]:
## Merge "peaks" if distance < disxy (spatial) & dist (time) 
disxy = 10
distime = 4
# Parameters: astime: save tif image as temporal stack (else slice stack)
temporal = True

for image in imname:
    print(image)
    if os.path.exists(image):
        if talkative:
            print( "Detecting events on movie "+str(image) )
        
        ## Detect events
        dexter.detect_events_onmovie( image, models=model, 
                                     cell_diameter=cell_diameter, extrusion_duration=extrusion_duration, 
                                     dxy=shift_xy, dz=shift_t, 
                                     group_size=groupsize )
          
        ## Save probability maps to file, to visualize the results
        if save_proba:
            dexter.write_probamaps(cat=None, astime=temporal)
        
        ## Save cleaned probability map(s) to file, keeping only events above the size and probability thresholds (for visualisation) 
        if save_proba_one:
            dexter.write_cleanedprobamap( cat=cat, volume_threshold=roi_volume, proba_threshold=roi_threshold, 
                                         disxy=disxy, distime=distime, astime=temporal )
        
        ## Look for centroid of "positive" volumes in the probability maps and convert it to a point event 
        ## Keep only ROIs that have a big enough volume ('volume_threshold') and with high enough probability ('proba_threshold') 
        # - put 0 and 0 to keep all detections
        if save_rois:
            dexter.get_rois(cat=cat, volume_threshold=roi_volume, proba_threshold=roi_threshold, disxy=disxy, dist=distime)